In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## ערכים עצמיים, וקטורים עצמיים, עקבה ודטרמיננטה

למטריצה $A$ מתקבלים **ערכים עצמיים** $\lambda_i$ ו־**וקטורים עצמיים** $v_i$ כך ש־

$$
A v_i = \lambda_i v_i .
$$

### אינטואיציה פיסיקלית
בפיסיקה, ערכים ווקטורים עצמיים מופיעים בהקשרים רבים:
- **מצבים נורמליים** במערכות לינאריות — למשל תנודות קטנות סביב נקודת שיווי משקל.
- **כיווני יציבות/אי־יציבות** במערכות דינמיות.
- **צירים עיקריים** של התפלגות או קירוב קוודרטי (principal axes).

* `np.linalg.eig(A)` מחזירה `(w, V)`:
  - `w` – מערך הערכים העצמיים.
  - `V` – מטריצה שבה כל עמודה היא וקטור עצמי מתאים.
* `np.linalg.eigvals(A)` מחזירה רק את הערכים העצמיים.

במקרה של מטריצה **סימטרית** (או הרמיטית ממשית), מתקיים:
* הערכים העצמיים ממשיים.
* הווקטורים העצמיים ניתנים לבחירה כאורתונורמליים (זווית ישרה ונורמה 1).
* לשימוש כזה עדיף `np.linalg.eigh`, המנצל את מבנה הסימטריה ומחזיר תוצאה יציבה יותר.

### קשרים שימושיים
* העקבה (trace) של מטריצה היא סכום הערכים העצמיים:

  $$
  \mathrm{trace}(A) = \sum_i \lambda_i
  $$

* הדטרמיננטה (det) היא מכפלת הערכים העצמיים:
  
  $$
  \det(A) = \prod_i \lambda_i
  $$

### ערכים עצמיים ווקטורים עצמיים עבור מטריצה סימטרית

כאשר $A$ היא מטריצה סימטרית (כלומר $A^T = A$), יש לה תכונות נוחות במיוחד:
* כל הערכים העצמיים שלה ממשיים.
* הווקטורים העצמיים שלה אורתונורמליים (מאונכים ובעלי אורך 1).

ב־NumPy עדיף להשתמש ב־`np.linalg.eigh` (במקום `eig`) עבור מטריצות כאלה, משום שהאלגוריתם מנצל את הסימטריה ומחזיר תוצאה יציבה וממוינת.

בדוגמה הזו נחשב:
* את הערכים העצמיים $w$ והווקטורים העצמיים $V$,
* את העקבה (`trace`) והדטרמיננטה (`det`) של $A$,
* ונראה שאכן מתקיים $\mathrm{trace}(A) = \sum_i \lambda_i$ ו־$\det(A) = \prod_i \lambda_i$.
  
בנוסף נבדוק:
* שהווקטורים העצמיים אורתונורמליים ($V^T V = I$),
* ושניתן לשחזר את $A$ מפירוק הערכים העצמיים:  
  $$
  A \approx V \, \mathrm{diag}(w) \, V^T
  $$



In [3]:
import numpy as np
# Symmetric example: prefer eigh
A = np.array([[4., 1., 0.],
              [1., 3., 1.],
              [0., 1., 2.]], dtype=float)

w, V = np.linalg.eigh(A)     # sorted ascending
trA = np.trace(A)
detA = np.linalg.det(A)

trace_vs_sum = trA, np.sum(w)
det_vs_prod   = detA, np.prod(w)

# Check orthonormality and reconstruction A ≈ V diag(w) V^T
Icheck = V.T @ V
Arec = V @ np.diag(w) @ V.T
np.allclose(Icheck, np.eye(3)), np.allclose(Arec, A), trace_vs_sum, det_vs_prod


(True,
 True,
 (np.float64(9.0), np.float64(9.000000000000002)),
 (np.float64(17.999999999999996), np.float64(18.000000000000007)))

### מטריצה לא סימטרית וערכים עצמיים מרוכבים

לא כל המטריצות הן סימטריות. כאשר $A$ אינה סימטרית, ייתכן שהערכים העצמיים שלה יהיו **מספרים מרוכבים**.

בדוגמה הזו נגדיר מטריצה $B$ שמתארת סיבוב במישור:

$$
B = \begin{bmatrix}
0 & -1 \\
1 & \;\;0
\end{bmatrix}
$$

המטריצה הזו לא סימטרית, ולכן הערכים העצמיים שלה יוצאים מרוכבים:

$$
\lambda = \pm i
$$

ב־NumPy נשתמש ב־`np.linalg.eig` כדי לקבל גם את הערכים העצמיים (שיכולים להיות מרוכבים) וגם את הווקטורים העצמיים המתאימים.

In [4]:
# Non-symmetric example: complex eigenvalues are possible
B = np.array([[0., -1.],
              [1.,  0.]])
wB, VB = np.linalg.eig(B)
wB, VB


(array([0.+1.j, 0.-1.j]),
 array([[0.70710678+0.j        , 0.70710678-0.j        ],
        [0.        -0.70710678j, 0.        +0.70710678j]]))